In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from datetime import datetime as dt

import plotly
plotly.tools.set_credentials_file(username='cathyhax', api_key='COw0zzFdojAcwZdDIdtf')

import plotly.plotly as py
import plotly.graph_objs as go

from ratings import Elo

In [2]:
fixtures = pd.read_csv('fixtures.csv')

In [3]:
#Use Champions League to test
fixtures_f = fixtures[(fixtures['league_name'] == 'Champions League') | (fixtures['league_name'] == 'Bundesliga')]

In [ ]:
champions_league_elo = Elo()

In [ ]:
champions_league_elo.calculate_elo(fixtures, 30, results_by_league = True)

Prepping data...
Starting calculations...


In [8]:
champions_league_elo.ratings_teams

,team_name,rating
team_id,,
3468.0,Real Madrid,1827.136538
83.0,Barcelona,1761.262759
503.0,Bayern München,1758.701080
625.0,Juventus,1729.242220
591.0,PSG,1693.034830
7980.0,Atlético Madrid,1688.390290
18.0,Chelsea,1647.536898
19.0,Arsenal,1643.985019
14.0,Manchester United,1639.765586


In [23]:
team_fixture_r = champions_league_elo.ratings_teams_fixtures

In [24]:
fixtures = fixtures.join(champions_league_elo.ratings_fixtures)

In [28]:
RM_fixture_r = team_fixture_r[team_fixture_r['team_name'] == 'Barcelona']

In [32]:
teams = team_fixture_r.team_name.unique()

In [46]:
data = []

for team in teams:
    team_data = team_fixture_r[team_fixture_r['team_name'] == team]
    team_trace = go.Scatter(
        x = team_data.order,
        y = team_data.team_r,
        name = team,
        opacity = 0.8)
    data.append(team_trace)

In [47]:
fig = dict(data = data)
py.iplot(data)